# FLAI AI Map Database

In [1]:
# Import libraries
from dotenv import load_dotenv
import os
import json
import psycopg2
import openai

In [2]:
# Environment variables
load_dotenv()

PG_HOST = os.getenv("POSTGRES_HOST")
PG_DB = os.getenv("POSTGRES_DB")
PG_USER = os.getenv("POSTGRES_USER")
PG_PASS = os.getenv("POSTGRES_PASSWORD")


In [3]:
# Databes connection
conn = psycopg2.connect(
    host=PG_HOST,
    database=PG_DB,
    user=PG_USER,
    password=PG_PASS
)

def execute_query(conn, query, params=None, fetch=False):
    cur = conn.cursor()
    """Execute a SQL query with optional parameters."""
    try:
        if params:
            cur.execute(query, params)
        else:
            cur.execute(query)
        if fetch:
            result = cur.fetchall()
            return result
        else:
            conn.commit()
            return cur.rowcount
    except Exception as e:
        print(f"Error executing query: {e}")
        conn.rollback()
        return None
    finally:
        cur.close()


## Populate States

In [4]:
def populate_states(connection, json_file_path):
    """
    Reads a GeoJSON file of states and inserts it into the flai.TCLStates table.
    """
    print(f"Starting data load from {json_file_path}...")

    # Check if the file exists
    if not os.path.exists(json_file_path):
        print(f"Error: File {json_file_path} not found.")
        return

    # Load the JSON file
    with open(json_file_path, 'r') as f:
        data = json.load(f)

    features = data.get('features', [])
    if not features:
        print("The JSON file does not contain 'features' or the list is empty.")
        return

    insert_query = """
        INSERT INTO flai.TCLStates (idState, State, geometry)
        VALUES (%s, %s, ST_Transform(ST_GeomFromGeoJSON(%s), 5070))
        ON CONFLICT (idState) DO NOTHING;
    """
    
    successful_inserts = 0
    for feature in features:
        try:
            state_id = feature.get('id')
            state_name = feature.get('properties', {}).get('name')
            geometry_obj = feature.get('geometry')

            if not all([state_id, state_name, geometry_obj]):
                print(f"Warning: Skipping record due to incomplete data: {feature}")
                continue

            # The geometry object must be passed as a string to the ST_GeomFromGeoJSON function
            geometry_str = json.dumps(geometry_obj)
            
            params = (state_id, state_name, geometry_str)
            
            print(f"Processing: {state_name} ({state_id})...")

            rows_affected = execute_query(connection, insert_query, params)
            if rows_affected is not None and rows_affected > 0:
                successful_inserts += 1

        except Exception as e:
            print(f"Error processing state {state_name}: {e}")

    print("-" * 30)
    print(f"Process completed. Inserted {successful_inserts} new records.")
    print(f"Total states processed: {len(features)}.")

In [5]:
populate_states(conn, "us-states.json")

Starting data load from us-states.json...
Processing: Alabama (AL)...
Processing: Alaska (AK)...
Processing: Arizona (AZ)...
Processing: Arkansas (AR)...
Processing: California (CA)...
Processing: Colorado (CO)...
Processing: Connecticut (CT)...
Processing: Delaware (DE)...
Processing: Florida (FL)...
Processing: Georgia (GA)...
Processing: Hawaii (HI)...
Processing: Idaho (ID)...
Processing: Illinois (IL)...
Processing: Indiana (IN)...
Processing: Iowa (IA)...
Processing: Kansas (KS)...
Processing: Kentucky (KY)...
Processing: Louisiana (LA)...
Processing: Maine (ME)...
Processing: Maryland (MD)...
Processing: Massachusetts (MA)...
Processing: Michigan (MI)...
Processing: Minnesota (MN)...
Processing: Mississippi (MS)...
Processing: Missouri (MO)...
Processing: Montana (MT)...
Processing: Nebraska (NE)...
Processing: Nevada (NV)...
Processing: New Hampshire (NH)...
Processing: New Jersey (NJ)...
Processing: New Mexico (NM)...
Processing: New York (NY)...
Processing: North Carolina (NC

In [6]:
# Dictionary to map state FIPS codes to their 2-letter abbreviations
FIPS_TO_STATE_ABBR = {
    '01': 'AL', '02': 'AK', '04': 'AZ', '05': 'AR', '06': 'CA', '08': 'CO', '09': 'CT', '10': 'DE',
    '11': 'DC', '12': 'FL', '13': 'GA', '15': 'HI', '16': 'ID', '17': 'IL', '18': 'IN', '19': 'IA',
    '20': 'KS', '21': 'KY', '22': 'LA', '23': 'ME', '24': 'MD', '25': 'MA', '26': 'MI', '27': 'MN',
    '28': 'MS', '29': 'MO', '30': 'MT', '31': 'NE', '32': 'NV', '33': 'NH', '34': 'NJ', '35': 'NM',
    '36': 'NY', '37': 'NC', '38': 'ND', '39': 'OH', '40': 'OK', '41': 'OR', '42': 'PA', '44': 'RI',
    '45': 'SC', '46': 'SD', '47': 'TN', '48': 'TX', '49': 'UT', '50': 'VT', '51': 'VA', '53': 'WA',
    '54': 'WV', '55': 'WI', '56': 'WY',
    # Territories
    '60': 'AS', '66': 'GU', '69': 'MP', '72': 'PR', '78': 'VI'
}


## Populate Counties

In [7]:
def populate_counties(connection, geojson_file_path):
    """
    Reads a GeoJSON file of counties and inserts them into the flai.TCLCounties table.
    """
    print(f"Starting data load from {geojson_file_path}...")

    if not os.path.exists(geojson_file_path):
        print(f"Error: File {geojson_file_path} not found.")
        return

    with open(geojson_file_path, 'r') as f:
        data = json.load(f)

    features = data.get('features', [])
    if not features:
        print("The GeoJSON file does not contain 'features' or the list is empty.")
        return

    # Use a CTE to transform the geometry only once.
    # The area is calculated in square meters (due to SRID 5070) and converted to square miles.
    insert_query = """
        WITH prepped_geom AS (
            SELECT ST_Transform(ST_GeomFromGeoJSON(%s), 5070) AS geom
        )
        INSERT INTO flai.TCLCounties (fips_county_code, County, idState, areaSQMI, geometry)
        SELECT %s, %s, %s, ST_Area(geom) * 0.000000386102, geom
        FROM prepped_geom
        ON CONFLICT (fips_county_code) DO NOTHING;
    """
    
    successful_inserts = 0
    skipped_count = 0
    total_features = len(features)
    
    print(f"Found {total_features} counties to process.")

    for i, feature in enumerate(features):
        try:
            fips_code = feature.get('id')
            properties = feature.get('properties', {})
            state_fips = properties.get('STATE')
            county_name = properties.get('NAME')
            geometry_obj = feature.get('geometry')

            if not all([fips_code, state_fips, county_name, geometry_obj]):
                print(f"\nWarning: Skipping record due to incomplete data: {feature}")
                skipped_count += 1
                continue
            
            # Map the state FIPS code to its abbreviation
            state_abbr = FIPS_TO_STATE_ABBR.get(state_fips)
            if not state_abbr:
                skipped_count += 1
                continue

            geometry_str = json.dumps(geometry_obj)
            
            # The order of parameters must match the %s placeholders in the query
            params = (geometry_str, fips_code, county_name, state_abbr)
            
            rows_affected = execute_query(connection, insert_query, params)
            if rows_affected is None:
                # This means execute_query caught an error (like the foreign key violation)
                #print(f"\nWarning: Skipping county '{county_name}' due to an error during insertion.")
                skipped_count += 1
            elif rows_affected > 0:
                # This means the insert was successful
                successful_inserts += 1

            # Print progress indicator without a newline
            progress = (i + 1) / total_features * 100
            print(f"\rProcessing: {i+1}/{total_features} ({progress:.2f}%)", end="", flush=True)

        except Exception as e:
            print(f"\nError processing county {county_name} ({fips_code}): {e}")
            skipped_count += 1
    
    print("\n" + "-" * 30)
    print("Process completed.")
    print(f"Total counties processed: {total_features}.")
    print(f"Successfully inserted: {successful_inserts} new records.")
    print(f"Skipped records: {skipped_count}.")


In [8]:
populate_counties(conn, "counties.geojson")

Starting data load from counties.geojson...
Found 3221 counties to process.
Processing: 625/3221 (19.40%)Error executing query: insert or update on table "tclcounties" violates foreign key constraint "tclcounties_idstate_fkey"
DETAIL:  Key (idstate)=(PR) is not present in table "tclstates".

Processing: 696/3221 (21.61%)Error executing query: insert or update on table "tclcounties" violates foreign key constraint "tclcounties_idstate_fkey"
DETAIL:  Key (idstate)=(PR) is not present in table "tclstates".

Processing: 697/3221 (21.64%)Error executing query: insert or update on table "tclcounties" violates foreign key constraint "tclcounties_idstate_fkey"
DETAIL:  Key (idstate)=(PR) is not present in table "tclstates".

Processing: 698/3221 (21.67%)Error executing query: insert or update on table "tclcounties" violates foreign key constraint "tclcounties_idstate_fkey"
DETAIL:  Key (idstate)=(PR) is not present in table "tclstates".

Processing: 699/3221 (21.70%)Error executing query: inse

## Populate Flood Events

In [9]:
def get_event_type_map(connection):
    """
    Queries the TCLEventTypes table and returns a dictionary mapping
    event type names to their primary key IDs.
    Example: {'Flash Flood': 1, 'Flood': 2}
    """
    print("Fetching event type mapping from the database...")
    query = "SELECT EventType, idEventType FROM flai.TCLEventTypes;"
    rows = execute_query(connection, query, fetch=True)
    if rows:
        event_map = {row[0]: row[1] for row in rows}
        print(f"Mapping successful: {event_map}")
        return event_map
    else:
        print("Error: Could not retrieve event types from the database.")
        return None

In [10]:
def populate_flood_events(connection, root_directory):
    """
    Walks through a directory, reads GeoJSON flood event files, and populates
    the TBLFloodEvents table.
    """
    event_type_map = get_event_type_map(connection)
    if not event_type_map:
        return # Stop if we can't get the event type IDs

    insert_query = """
        INSERT INTO flai.TBLFloodEvents (idEventType, beginDate, warning_zone, geometry)
        VALUES (%s, %s, %s, ST_SetSRID(ST_GeomFromGeoJSON(%s), 4326))
        RETURNING 1; 
    """

    successful_inserts = 0
    skipped_records = 0
    total_records_processed = 0

    print(f"\nStarting to process files in '{root_directory}'...")

    for dirpath, _, filenames in os.walk(root_directory):
        for filename in filenames:
            # You can add a check here like `if not filename.endswith('.geojson'): continue`
            # but for now, we'll try to process all files found.
            file_path = os.path.join(dirpath, filename)
            
            try:
                with open(file_path, 'r') as f:
                    data = json.load(f)
                
                features = data.get('features', [])
                print(f"\nProcessing file: {file_path} ({len(features)} events)")

                for feature in features:
                    total_records_processed += 1
                    properties = feature.get('properties', {})
                    
                    # 1. Get Event Type ID from our map
                    event_type_name = properties.get('EVENT_TYPE')
                    event_type_id = event_type_map.get(event_type_name)
                    
                    # 2. Get and format the date
                    begin_date_str = properties.get('BEGIN_DATE', '').split('T')[0]

                    # 3. Get the warning zone
                    warning_zone_name = properties.get('CZ_NAME_STR')
                    
                    # 4. Get the geometry object
                    geometry_obj = feature.get('geometry')

                    # Validate data before trying to insert
                    if not all([event_type_id, begin_date_str, geometry_obj]):
                        print(f"\nWarning: Skipping record with incomplete data: {properties}")
                        skipped_records += 1
                        continue

                    geometry_str = json.dumps(geometry_obj)
                    params = (event_type_id, begin_date_str, warning_zone_name, geometry_str)

                    result = execute_query(connection, insert_query, params)
                    
                    if result:
                        successful_inserts += 1
                    else:
                        # This catches both database errors and ON CONFLICT skips if you add one
                        skipped_records += 1
                
                # Progress indicator
                print(f"-> File complete. Total inserted so far: {successful_inserts}")

            except json.JSONDecodeError:
                print(f"\nWarning: Skipping non-JSON file or file with invalid format: {file_path}")
            except Exception as e:
                print(f"\nAn unexpected error occurred while processing {file_path}: {e}")

    print("\n" + "=" * 40)
    print("Flood Event Population Process Completed")
    print("=" * 40)
    print(f"Total records found in files: {total_records_processed}")
    print(f"Successfully inserted:        {successful_inserts} new records")
    print(f"Skipped records:              {skipped_records}")


In [11]:
populate_flood_events(conn, "flood-data")

Fetching event type mapping from the database...
Mapping successful: {'Flash Flood': 1, 'Flood': 2}

Starting to process files in 'flood-data'...

An unexpected error occurred while processing flood-data/.DS_Store: 'utf-8' codec can't decode byte 0x87 in position 23: invalid start byte

Processing file: flood-data/flash-flood-events/AL_Flood_Events_2020.geojson (116 events)
-> File complete. Total inserted so far: 116

Processing file: flood-data/flash-flood-events/AL_Flood_Events_2021.geojson (166 events)
-> File complete. Total inserted so far: 282

Processing file: flood-data/flash-flood-events/AL_Flood_Events_2023.geojson (84 events)
-> File complete. Total inserted so far: 366

Processing file: flood-data/flash-flood-events/AL_Flood_Events_2022.geojson (115 events)
-> File complete. Total inserted so far: 481

Processing file: flood-data/flash-flood-events/AL_Flood_Events_2024.geojson (77 events)
-> File complete. Total inserted so far: 558

Processing file: flood-data/flash-flood

## Populate SVI 2022

In [12]:
def get_svi_theme_map(connection):
    """
    Queries the TCLSVIThemes table and returns two dictionaries:
    1. Mapping theme name to its ID.
    2. Mapping theme number (1-4) to its ID.
    """
    print("Fetching SVI theme mappings...")
    query = "SELECT idSVITheme, Theme FROM flai.TCLSVIThemes;"
    rows = execute_query(connection, query, fetch=True)
    if not rows:
        raise Exception("Could not retrieve SVI themes from the database.")
    
    name_to_id = {row[1]: row[0] for row in rows}
    
    # Create a mapping from the number in the JSON key (e.g., '1' in RPL_THEME1) to the theme ID
    # This relies on the themes being inserted in the correct order, which your script does.
    # A more robust solution would be to add a 'theme_number' column to TCLSVIThemes.
    num_to_id = {}
    for name, theme_id in name_to_id.items():
        if 'Socioeconomic' in name: num_to_id[1] = theme_id
        elif 'Household' in name: num_to_id[2] = theme_id
        elif 'Racial' in name: num_to_id[3] = theme_id
        elif 'Housing' in name: num_to_id[4] = theme_id
            
    print("Theme mapping successful.")
    return name_to_id, num_to_id

def get_svi_variable_map(connection):
    """
    Queries the TCLSVIVariables table and returns a dictionary mapping
    the variable code (e.g., 'EPL_POV150_state') to its IDs.
    Example: {'EPL_POV150_state': {'var_id': 1, 'theme_id': 1}}
    """
    print("Fetching SVI variable mappings...")
    query = "SELECT idSVIVariable, idSVITheme, Code FROM flai.TCLSVIVariables;"
    rows = execute_query(connection, query, fetch=True)
    if not rows:
        raise Exception("Could not retrieve SVI variables from the database.")
    
    var_map = {row[2]: {'var_id': row[0], 'theme_id': row[1]} for row in rows}
    print("Variable mapping successful.")
    return var_map

In [13]:
def populate_svi_data(connection, geojson_file_path, release_year):
    """
    Reads a GeoJSON file with SVI data and populates the flai.TBLSVI table.
    Each feature in the GeoJSON is unpivoted into 20 rows in the database.
    """
    try:
        theme_name_to_id, theme_num_to_id = get_svi_theme_map(connection)
        variable_map = get_svi_variable_map(connection)
    except Exception as e:
        print(f"Error during pre-fetch: {e}")
        return

    insert_query = """
        INSERT INTO flai.TBLSVI (
            fips_county_code, release_year, idSVITheme, idSVIVariable,
            overallNational, overallState, SVIValue
        )
        VALUES (%s, %s, %s, %s, %s, %s, %s); 
    """ # No RETURNING needed, we'll count in Python

    if not os.path.exists(geojson_file_path):
        print(f"Error: File {geojson_file_path} not found.")
        return

    with open(geojson_file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    features = data.get('features', [])
    total_features = len(features)
    successful_inserts = 0
    skipped_records = 0
    
    print(f"\nFound {total_features} counties to process for year {release_year}...")

    for i, feature in enumerate(features):
        properties = feature.get('properties', {})
        fips = properties.get('FIPS')
        
        # SVI values can sometimes be -999 for "no data". We'll treat them as NULL.
        overall_national = properties.get('RPL_THEMES_national')
        if overall_national == -999: overall_national = None
        
        overall_state = properties.get('RPL_THEMES_state')
        if overall_state == -999: overall_state = None

        if not fips:
            skipped_records += 20 # Skip all 20 potential records for this feature
            continue

        # --- 1. Insert the 4 Theme-level records ---
        for theme_num, theme_id in theme_num_to_id.items():
            json_key = f'RPL_THEME{theme_num}_state'
            svi_value = properties.get(json_key)
            if svi_value == -999: svi_value = None
            
            params = (
                fips, release_year, theme_id, None, # idSVIVariable is NULL for themes
                overall_national, overall_state, svi_value
            )
            # Using basic try/except for counting, as execute_query handles errors
            try:
                execute_query(connection, insert_query, params)
                successful_inserts += 1
            except:
                skipped_records += 1

        # --- 2. Insert the 16 Variable-level records ---
        for code, ids in variable_map.items():
            svi_value = properties.get(code)
            if svi_value == -999: svi_value = None

            params = (
                fips, release_year, ids['theme_id'], ids['var_id'],
                overall_national, overall_state, svi_value
            )
            try:
                execute_query(connection, insert_query, params)
                successful_inserts += 1
            except:
                skipped_records += 1

        if (i + 1) % 50 == 0: # Progress update every 50 counties
             print(f"\rProcessed {i+1}/{total_features} counties...", end="", flush=True)

    print("\n" + "=" * 40)
    print("SVI Data Population Process Completed")
    print("=" * 40)
    print(f"Total counties processed:     {total_features}")
    print(f"Successfully inserted rows:   {successful_inserts}")
    print(f"Skipped/Failed rows:          {skipped_records}")



In [14]:
populate_svi_data(conn, "alabama_svi_tracts_master.geojson", 2022)

Fetching SVI theme mappings...
Theme mapping successful.
Fetching SVI variable mappings...
Variable mapping successful.

Found 67 counties to process for year 2022...
Processed 50/67 counties...
SVI Data Population Process Completed
Total counties processed:     67
Successfully inserted rows:   1340
Skipped/Failed rows:          0


## Populate Precipitation Data

In [15]:
def populate_precipitation_data(connection, root_directory):
    """
    Walks through a directory structure of year/month.geojson, reads precipitation
    data, and populates the flai.TBLMonthlyPrecipitation table.
    """
    # Dictionary to convert month names (from filenames) to numbers
    month_map = {
        'january': 1, 'february': 2, 'march': 3, 'april': 4, 'may': 5, 'june': 6,
        'july': 7, 'august': 8, 'september': 9, 'october': 10, 'november': 11, 'december': 12
    }

    # This query will insert a new record or update an existing one if the
    # combination of fips_code, year, and month already exists.
    insert_query = """
        INSERT INTO flai.TBLMonthlyPrecipitation (fips_county_code, year, month, totalPrecipitation_mm)
        VALUES (%s, %s, %s, %s)
        ON CONFLICT (fips_county_code, year, month) DO UPDATE SET
            totalPrecipitation_mm = EXCLUDED.totalPrecipitation_mm;
    """

    successful_ops = 0
    skipped_records = 0
    
    print(f"Starting to process precipitation files in '{root_directory}'...")

    # os.walk is perfect for navigating the directory tree
    for dirpath, _, filenames in os.walk(root_directory):
        for filename in filenames:
            if not filename.endswith('.geojson'):
                continue

            file_path = os.path.join(dirpath, filename)

            try:
                # Extract year from the parent directory's name
                year = int(os.path.basename(dirpath))
                
                # Extract month name from filename and convert to number
                month_name = os.path.splitext(filename)[0].lower()
                month_num = month_map.get(month_name)

                if not month_num:
                    print(f"\nWarning: Skipping file with non-month name: {filename}")
                    continue
                
                print(f"\nProcessing file: {file_path} (Year: {year}, Month: {month_num})")

                with open(file_path, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                
                features = data.get('features', [])
                for feature in features:
                    properties = feature.get('properties', {})
                    fips_raw = properties.get('fips_code')
                    precip_mm = properties.get('total_precipitation_mm')

                    if not fips_raw or precip_mm is None:
                        skipped_records += 1
                        continue
                    
                    # Clean the FIPS code by removing the "FIPS:" prefix
                    fips_clean = fips_raw.replace('FIPS:', '').strip()
                    
                    if len(fips_clean) != 5:
                        print(f"Warning: Skipping malformed FIPS code '{fips_clean}'")
                        skipped_records += 1
                        continue

                    params = (fips_clean, year, month_num, precip_mm)
                    
                    # execute_query will return 1 for an INSERT or UPDATE
                    result = execute_query(connection, insert_query, params)
                    if result is not None:
                        successful_ops += 1
                    else:
                        skipped_records += 1

            except ValueError:
                # This catches cases where the directory name is not a valid integer year
                print(f"\nWarning: Skipping directory with non-year name: {dirpath}")
            except Exception as e:
                print(f"\nAn unexpected error occurred with file {file_path}: {e}")

    print("\n" + "=" * 40)
    print("Precipitation Data Population Process Completed")
    print("=" * 40)
    print(f"Successful inserts/updates: {successful_ops}")
    print(f"Skipped records (bad data): {skipped_records}")


In [16]:
populate_precipitation_data(conn, "precipitation-data")

Starting to process precipitation files in 'precipitation-data'...

Processing file: precipitation-data/2024/november.geojson (Year: 2024, Month: 11)

Processing file: precipitation-data/2024/january.geojson (Year: 2024, Month: 1)

Processing file: precipitation-data/2024/march.geojson (Year: 2024, Month: 3)

Processing file: precipitation-data/2024/may.geojson (Year: 2024, Month: 5)

Processing file: precipitation-data/2024/june.geojson (Year: 2024, Month: 6)

Processing file: precipitation-data/2024/august.geojson (Year: 2024, Month: 8)

Processing file: precipitation-data/2024/december.geojson (Year: 2024, Month: 12)

Processing file: precipitation-data/2024/february.geojson (Year: 2024, Month: 2)

Processing file: precipitation-data/2024/april.geojson (Year: 2024, Month: 4)

Processing file: precipitation-data/2024/october.geojson (Year: 2024, Month: 10)

Processing file: precipitation-data/2024/july.geojson (Year: 2024, Month: 7)

Processing file: precipitation-data/2024/september

In [17]:
# Close the database connection when done
if conn:
    conn.close()
    print("\nDatabase connection closed.")


Database connection closed.
